<a href="https://colab.research.google.com/github/rinrin528/AIFFEL-1st-Hackathon/blob/main/%ED%95%B4%EC%BB%A4%ED%86%A41_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble, neighbors, linear_model, metrics, preprocessing
from datetime import datetime
import glob, re
import time, datetime
from datetime import timedelta
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = {
    'tra': pd.read_csv('/content/drive/MyDrive/data/air_visit_data.csv'),
    'as': pd.read_csv('/content/drive/MyDrive/data/air_store_info.csv'),
    'hs': pd.read_csv('/content/drive/MyDrive/data/hpg_store_info.csv'),
    'ar': pd.read_csv('/content/drive/MyDrive/data/air_reserve.csv'),
    'hr': pd.read_csv('/content/drive/MyDrive/data/hpg_reserve.csv'),
    'id': pd.read_csv('/content/drive/MyDrive/data/store_id_relation.csv'),
    'tes': pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv'),
    'hol': pd.read_csv('/content/drive/MyDrive/data/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])        # bring air id to hpg reserve data
data['hs'] = pd.merge(data['hs'], data['id'], how='inner', on=['hpg_store_id'])        # bring air id to hpg stores





for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_hour'] = data[df]['visit_datetime'].dt.hour
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date


    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_hour'] = data[df]['reserve_datetime'].dt.hour
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date

    data[df+'_hour'] = data[df]#keeping original

    #calculate reserve time difference and summarizing ar,hr to date
    data[df]['reserve_datetime_diff'] = data[df].apply(
        lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    display(data[df])
    data[df] = data[df].groupby(
        ['air_store_id', 'visit_datetime'], as_index=False)[[
            'reserve_datetime_diff', 'reserve_visitors'
        ]].sum().rename(columns={
            'visit_datetime': 'visit_date'
        })




data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])    # 날짜 데이터타입으로 변경
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek              # 요일 컬럼 생성
data['tra']['year'] = data['tra']['visit_date'].dt.year                  # 년도 컬럼 생성
data['tra']['month'] = data['tra']['visit_date'].dt.month                # month 컬럼 생성
data['tra']['dow_name'] = data['tra']['visit_date'].dt.day_name()

data['tra']['visit_date'] = data['tra']['visit_date'].dt.date
data['tra']['year_mth'] = data['tra']['year'].astype(str)+'-'+data['tra']['month'].astype(str)





data['tes']['visit_date'] = data['tes']['id'].map(
    lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(
    lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['dow_name'] = data['tes']['visit_date'].dt.day_name()
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month

data['tes']['visit_date'] = data['tes']['visit_date'].dt.date
data['tes']['year_mth'] = data['tes']['year'].astype(str)+'-'+data['tes']['month'].astype(str)






#extract unique stores based on test data and populate dow 1 to 6
unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat(
    [
        pd.DataFrame({
            'air_store_id': unique_stores,
            'dow': [i] * len(unique_stores)
        }) for i in range(7)
    ],
    axis=0,
    ignore_index=True).reset_index(drop=True)





# mapping train data dow to stores(test data) - min, mean, median, max, count 
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].min().rename(columns={
        'visitors': 'min_visitors' })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].mean().rename(columns={
        'visitors': 'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].median().rename(columns={
        'visitors': 'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].max().rename(columns={
        'visitors': 'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].count().rename(columns={
        'visitors': 'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id'])



,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_hour,reserve_hour,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1,19,16,0
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3,19,19,0
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6,19,19,0
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2,20,16,0
4,air_db80363d35f10926,2016-01-01,2016-01-01,5,20,1,0
...,...,...,...,...,...,...,...
92373,air_db4b38ebe7a7ceff,2017-05-29,2017-04-17,6,18,19,42
92374,air_1033310359ceeac1,2017-05-30,2017-04-19,8,23,17,41
92375,air_877f79706adbfb06,2017-05-31,2017-04-12,3,20,23,49
92376,air_900d755ebd2f7bbd,2017-05-31,2017-04-19,10,20,16,42


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,visit_hour,reserve_hour,reserve_datetime_diff
0,hpg_878cc70b1abc76f7,2016-01-01,2016-01-01,4,air_db80363d35f10926,19,15,0
1,hpg_878cc70b1abc76f7,2016-01-02,2016-01-02,2,air_db80363d35f10926,19,14,0
2,hpg_878cc70b1abc76f7,2016-01-03,2016-01-02,6,air_db80363d35f10926,18,20,1
3,hpg_878cc70b1abc76f7,2016-01-06,2016-01-04,3,air_db80363d35f10926,20,22,2
4,hpg_878cc70b1abc76f7,2016-01-11,2016-01-11,2,air_db80363d35f10926,18,14,0
...,...,...,...,...,...,...,...,...
28178,hpg_0f78ef8d1e4b23d8,2017-04-01,2017-03-10,2,air_84f6876ff7e83ae7,21,20,22
28179,hpg_0f78ef8d1e4b23d8,2017-04-02,2017-04-02,2,air_84f6876ff7e83ae7,17,14,0
28180,hpg_0f78ef8d1e4b23d8,2017-04-08,2017-04-03,3,air_84f6876ff7e83ae7,19,12,5
28181,hpg_0f78ef8d1e4b23d8,2017-04-08,2017-03-09,4,air_84f6876ff7e83ae7,20,7,30


In [3]:
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))


lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])



data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 



train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])



for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)
display(train)




train['total_reserv_sum'] = train['reserve_visitors_x'] + train['reserve_visitors_y']
train['total_reserv_mean'] = (train['reserve_visitors_x'] + train['reserve_visitors_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['reserve_datetime_diff_x'] + train['reserve_datetime_diff_y']) / 2

test['total_reserv_sum'] = test['reserve_visitors_x'] + test['reserve_visitors_y']
test['total_reserv_mean'] = (test['reserve_visitors_x'] + test['reserve_visitors_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['reserve_datetime_diff_x'] + test['reserve_datetime_diff_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

,air_store_id,visit_date,visitors,dow,year,month,dow_name,year_mth,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,id
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,Wednesday,2016-1,6,0,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-13
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,Thursday,2016-1,4,0,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-14
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,Friday,2016-1,0,0,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-15
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,Saturday,2016-1,2,0,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-16
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,Monday,2016-1,1,0,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_ba937bf13d40fb24_2016-01-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6,1,2017,4,Tuesday,2017-4,5,0,2.0,5.615385,6.0,11.0,39.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_24e8414b9b07decb_2017-04-18
252104,air_24e8414b9b07decb,2017-04-19,6,2,2017,4,Wednesday,2017-4,6,0,2.0,6.575000,6.0,25.0,40.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_24e8414b9b07decb_2017-04-19
252105,air_24e8414b9b07decb,2017-04-20,7,3,2017,4,Thursday,2017-4,4,0,2.0,5.394737,6.0,8.0,38.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_24e8414b9b07decb_2017-04-20
252106,air_24e8414b9b07decb,2017-04-21,8,4,2017,4,Friday,2017-4,0,0,2.0,5.738095,6.0,9.0,42.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,air_24e8414b9b07decb_2017-04-21


In [4]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [5]:
train

,air_store_id,visit_date,visitors,dow,year,month,dow_name,year_mth,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,air_genre_name,air_area_name,latitude,longitude,air_genre_name0,air_area_name0,air_genre_name1,air_area_name1,air_genre_name2,air_area_name2,air_genre_name3,air_area_name3,air_genre_name4,air_area_name4,air_genre_name5,air_area_name5,air_genre_name6,air_area_name6,air_genre_name7,air_area_name7,air_genre_name8,air_area_name8,air_genre_name9,air_area_name9,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,date_int,var_max_lat,var_max_long,lon_plus_lat,air_store_id2
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,Wednesday,2016-1,6,0,7.0,23.843750,25.0,57.0,64.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_ba937bf13d40fb24_2016-01-13,-1.0,-1.0,-1.0,20160113,8.362564,4.521799,175.409667,603
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,Thursday,2016-1,4,0,2.0,20.292308,21.0,54.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_ba937bf13d40fb24_2016-01-14,-1.0,-1.0,-1.0,20160114,8.362564,4.521799,175.409667,603
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,Friday,2016-1,0,0,4.0,34.738462,35.0,61.0,65.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_ba937bf13d40fb24_2016-01-15,-1.0,-1.0,-1.0,20160115,8.362564,4.521799,175.409667,603
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,Saturday,2016-1,2,0,6.0,27.651515,27.0,53.0,66.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_ba937bf13d40fb24_2016-01-16,-1.0,-1.0,-1.0,20160116,8.362564,4.521799,175.409667,603
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,Monday,2016-1,1,0,2.0,13.754386,12.0,34.0,57.0,4.0,62.0,35.658068,139.751599,4.0,7.0,7.0,6.0,0.0,26.0,0.0,6.0,0.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_ba937bf13d40fb24_2016-01-18,-1.0,-1.0,-1.0,20160118,8.362564,4.521799,175.409667,603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-18,6,1,2017,4,Tuesday,2017-4,5,0,2.0,5.615385,6.0,11.0,39.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_24e8414b9b07decb_2017-04-18,-1.0,-1.0,-1.0,20170418,8.367414,4.562362,175.364254,98
252104,air_24e8414b9b07decb,2017-04-19,6,2,2017,4,Wednesday,2017-4,6,0,2.0,6.575000,6.0,25.0,40.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_24e8414b9b07decb_2017-04-19,-1.0,-1.0,-1.0,20170419,8.367414,4.562362,175.364254,98
252105,air_24e8414b9b07decb,2017-04-20,7,3,2017,4,Thursday,2017-4,4,0,2.0,5.394737,6.0,8.0,38.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_24e8414b9b07decb_2017-04-20,-1.0,-1.0,-1.0,20170420,8.367414,4.562362,175.364254,98
252106,air_24e8414b9b07decb,2017-04-21,8,4,2017,4,Friday,2017-4,0,0,2.0,5.738095,6.0,9.0,42.0,11.0,72.0,35.653217,139.711036,11.0,7.0,0.0,6.0,0.0,38.0,0.0,6.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,air_24e8414b9b07decb_2017-04-21,-1.0,-1.0,-1.0,20170421,8.367414,4.562362,175.364254,98


In [6]:
train = train.drop(['dow_name', 'year_mth'], axis=1)

In [12]:
col.remove('dow_name')

In [8]:
col

['dow',
 'year',
 'month',
 'dow_name',
 'day_of_week',
 'holiday_flg',
 'min_visitors',
 'mean_visitors',
 'median_visitors',
 'max_visitors',
 'count_observations',
 'air_genre_name',
 'air_area_name',
 'latitude',
 'longitude',
 'air_genre_name0',
 'air_area_name0',
 'air_genre_name1',
 'air_area_name1',
 'air_genre_name2',
 'air_area_name2',
 'air_genre_name3',
 'air_area_name3',
 'air_genre_name4',
 'air_area_name4',
 'air_genre_name5',
 'air_area_name5',
 'air_genre_name6',
 'air_area_name6',
 'air_genre_name7',
 'air_area_name7',
 'air_genre_name8',
 'air_area_name8',
 'air_genre_name9',
 'air_area_name9',
 'reserve_datetime_diff_x',
 'reserve_visitors_x',
 'reserve_datetime_diff_y',
 'reserve_visitors_y',
 'total_reserv_sum',
 'total_reserv_mean',
 'total_reserv_dt_diff_mean',
 'date_int',
 'var_max_lat',
 'var_max_long',
 'lon_plus_lat',
 'air_store_id2']

In [13]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=280, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =12)



model1.fit(train[col], np.log1p(train['visitors'].values))
model2.fit(train[col], np.log1p(train['visitors'].values))
model3.fit(train[col], np.log1p(train['visitors'].values))



preds1 = model1.predict(train[col])
preds2 = model2.predict(train[col])
preds3 = model3.predict(train[col])




print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds2))
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds3))
preds1 = model1.predict(test[col])
preds2 = model2.predict(test[col])
preds3 = model3.predict(test[col])

test['visitors'] = 0.3*preds1+0.3*preds2+0.4*preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
del train; del data;

[11:40:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE GradientBoostingRegressor:  0.3488616404368805
RMSE KNeighborsRegressor:  0.4196828814038707
RMSE XGBRegressor:  0.3994038900459509


In [36]:
np.set_printoptions(threshold=np.inf)

print(preds1)
print(preds2)
print(preds3)

[ 0.62273054  3.10335739  3.51484017  3.23295782  3.57298565  3.76213759  2.56109226  0.62273054  3.14907814  3.55995503  2.51405684  2.96485518  2.77606273  2.44461613  0.56774885  3.14907814  3.55995503  3.2802718   3.60040445  3.81085255  2.44461613  0.56774885  3.14907814  3.55995503  3.2802718   3.60040445  3.81085255  2.44461613  0.56774885  3.14907814  3.55995503  3.2802718   3.60040445  3.81085255  2.44461613  0.56774885  3.14907814  3.55995503  3.2802718  -0.15396549  1.20656818  1.48739841  1.95109894  1.45428237  2.05859986  1.84011055 -0.15396549  1.27681209  1.49491949  2.26495962  1.38099083  2.6118227   1.70118576 -0.15423938  1.24835158  1.48369082  1.8889964   1.08165784  2.05889627  2.77933099 -0.15423938  1.24835158  1.50788592  1.8889964   1.08165784  2.05889627  1.70118576 -0.15423938  1.24835158  1.50788592  1.8889964   1.08165784  2.05889627  1.70118576 -0.15423938  1.24835158  1.50788592  1.8889964   2.39770095  2.21346976  2.2690193   2.23362551  0.42117204  1.

In [37]:
print(preds1.shape)
print(preds2.shape)
print(preds3.shape)

(32019,)
(32019,)
(32019,)


In [ ]:
test['visitors'] = preds3
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()

In [26]:
date_info = pd.read_csv('/content/drive/MyDrive/data/date_info.csv')
air_visit_data = pd.read_csv('/content/drive/MyDrive/data/air_visit_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/data/sample_submission.csv')

In [27]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('/content/data/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

sub_merge['visitors'] = 0.7*sub_merge['visitors_x'] + 0.3*sub_merge['visitors_y']* 1.1
sub_merge[['id', 'visitors']].to_csv('submission.csv', index=False)

In [28]:
df = pd.read_csv('/content/submission.csv')
df

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.070428
1,air_00a91d42b08b08d9_2017-04-24,23.731100
2,air_00a91d42b08b08d9_2017-04-25,31.771895
3,air_00a91d42b08b08d9_2017-04-26,27.614394
4,air_00a91d42b08b08d9_2017-04-27,34.150450
...,...,...
32014,air_fff68b929994bfbd_2017-05-27,5.692376
32015,air_fff68b929994bfbd_2017-05-28,3.124394
32016,air_fff68b929994bfbd_2017-05-29,2.991914
32017,air_fff68b929994bfbd_2017-05-30,3.274818
